In [1]:
username = "tester"
password  = "password"
url = "bolt://localhost:7687"
database="ctgov"
node_label= "AdverseEvent", 
embedding_node_property="biobert_emb"
index_name = "adverse_event"
text_node_properties = ["term","organ_system"]
user_query = "Anaemia"

In [2]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding as LI_HF

from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import VectorStoreIndex, StorageContext


def completion_to_prompt(completion: str) -> str:
  return "[INST] {completion} [/INST] "

def messages_to_prompt(messages) -> str:
  messages_str = [str(x) + "\n" for x in messages]
  return "[INST] {completion} [/INST] "

lm = Ollama(model="mistral", temperature=0.7,  request_timeout=30, 
            completion_to_prompt=completion_to_prompt, messages_to_prompt=messages_to_prompt)

li_biobert = LI_HF(model_name="dmis-lab/biobert-base-cased-v1.1")

Settings.llm = lm
Settings.embed_model = li_biobert



c:\Users\joanp\Projects\ctogov_rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# neo4j_vector = Neo4jVectorStore(
#     username,
#     password,
#     url,
#     768,
#     node_label= node_label, 
#     embedding_node_property= embedding_node_property, 
#     text_node_property= text_node_properties,
#     index_name=index_name,
#     keyword_index_name= "adverse_event_kw",
#     hybrid_search=True
# )

# loaded_index = VectorStoreIndex.from_vector_store(neo4j_vector).as_query_engine()
# test= loaded_index.query(user_query)
# print(test)

In [4]:
graph_store = Neo4jGraphStore(username=username, 
                              password=password, 
                              url=url, 
                              database=database,
                              node_label= node_label, 
                              embedding_node_property= embedding_node_property, 
                              text_node_properties= text_node_properties,
                              index_name=index_name,
                              search_type="hybrid",
                              keyword_index_name= "adverse_event_kw",
                              )

storage_context = StorageContext.from_defaults(graph_store=graph_store)

from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
)

In [5]:
user_question =  "List the clinical studies that reported Anaemia adverse events"
response = query_engine.query(user_query)
print(response)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'AdverseEvent': expected "%", "(" or an identifier (line 1, column 14 (offset: 13))
"MATCH p=(n1:('AdverseEvent',))-[*1..2]->() WHERE toLower(n1.id) IN ['anaemia', 'anemia', 'disorder', 'deficiency', 'hemoglobin', 'microcytic anemia', 'iron-deficiency anemia', 'anemia', 'iron', 'microcytic', 'hemoglobin deficiency.', 'anaemia disorder']UNWIND relationships(p) AS rel WITH n1.id AS subj, p, apoc.coll.flatten(apoc.coll.toSet(collect([type(rel), endNode(rel).id]))) AS flattened_rels RETURN subj, collect(flattened_rels) AS flattened_rels LIMIT 30"
              ^}

In [ ]:
from langchain_community.vectorstores import Neo4jVector
from langchain_community.embeddings import HuggingFaceEmbeddings as LC_HF

lc_biobert = LC_HF(model_name="dmis-lab/biobert-base-cased-v1.1")

adverse_event = Neo4jVector.from_existing_graph(
    username=username, 
    password=password, 
    database=database,
    url=url,
    node_label= node_label, 
    embedding_node_property= embedding_node_property, 
    text_node_properties= text_node_properties,
    index_name=index_name,
    search_type="hybrid",
    keyword_index_name= "adverse_event_kw",
    embedding= lc_biobert,
    )

test = adverse_event.similarity_search_with_score(user_query, k=3)
print(test)

text2sql

In [ ]:
import os
from dotenv import load_dotenv
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core import SQLDatabase
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from sqlalchemy import create_engine
import pandas as pd
from tqdm import tqdm
from requests.exceptions import ReadTimeout, Timeout

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Possible solution to llama-index timeout
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import yaml

# Load the YAML file
with open( "./src/txt_2_sql/sql_queries_template.yaml", "r") as file:
    sql_queries_template = yaml.safe_load(file)

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

def completion_to_prompt(completion: str) -> str:
  return "[INST] {completion} [/INST] "

def messages_to_prompt(messages) -> str:
  messages_str = [str(x) + "\n" for x in messages]
  return "[INST] {completion} [/INST] "

lm = Ollama(model="mixtral", temperature=0.0,  request_timeout=100, 
            completion_to_prompt=completion_to_prompt, messages_to_prompt=messages_to_prompt)
Settings.llm = lm
Settings.embed_model="local"

In [ ]:
load_dotenv(".env")
AACT_USER = os.getenv("AACT_USER")
AACT_PWD = os.getenv("AACT_PWD")

In [ ]:
tables = [
    "browse_interventions",
    "sponsors",
    "detailed_descriptions",
    "facilities",
    "studies",
    "outcomes",
    "browse_conditions",
    "keywords",
    "eligibilities",
    "reported_events",
    "brief_summaries",
    "designs",
    "countries",
]

database = "aact"
host = "aact-db.ctti-clinicaltrials.org"
user = AACT_USER
password = AACT_PWD
port = 5432
db_uri = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"


In [ ]:
db_engine = create_engine(db_uri)
sql_db = SQLDatabase(db_engine, include_tables=tables)
query_engine = NLSQLTableQueryEngine(sql_database=sql_db)

In [ ]:
condition = 'Asthma'
intervention = 'Xhance'
nctId = 'NCT01164592' 

In [ ]:
sql_queries_template["query_1"]["question"].format(nctId=nctId)

In [ ]:
response = query_engine.query(sql_queries_template["query_1"]["question"].format(nctId=nctId))
print(response)
print("######")
print(response.metadata["sql_query"])
print("######")
print(response.metadata["result"])

In [ ]:
def run_llamaindex_eval(verbose:bool=True) -> pd.DataFrame:
    "Run txt-2-SQL evalution over a Llama-index SQL query engine"
    
    sql_eval_cols = ["question", "gold_std_query", "gold_std_answer", "llamaIndex_query", "llamaIndex_answer"]
    sql_eval_rows = list(sql_queries_template.keys())
    sql_eval = pd.DataFrame([], index=sql_eval_rows, columns=sql_eval_cols)

    for q, d in tqdm(sql_queries_template.items(), desc= "Evaluating llama index"):
        question = d["question"].format(nctId=nctId, condition=condition, intervention=intervention)
        sql_query = d["SQL"].format(nctId=nctId, condition=condition, intervention=intervention)
        
        if verbose:
            print(f"{q} : {question}")
        
        sql_eval.at[q, "question"] =  question
        sql_eval.at[q, "gold_std_query"] = sql_query
        
        # Get gold standard answer
        try :
            answer = sql_db.run_sql(sql_query)[0]
        except:
            answer = "No answer"
            
        sql_eval.at[q, "gold_std_answer"] =  answer
        
        # Get LlamaIndex SQL query and answer
        try:
            response = query_engine.query(question)
            sql_eval.at[q, "llamaIndex_query"] = response.metadata["sql_query"]
            sql_eval.at[q, "llamaIndex_answer"] = response.response
        except (ReadTimeout, Timeout, TimeoutError):
            if verbose:
                print("Time out!")
            sql_eval.at[q, "llamaIndex_query"] = "ReadTimeout"
            sql_eval.at[q, "llamaIndex_answer"] = "ReadTimeout"
        except Exception as e:
            if verbose:
                print(e)
            sql_eval.at[q, "llamaIndex_query"] = e
            sql_eval.at[q, "llamaIndex_answer"] = e
    
    return sql_eval

In [ ]:
sql_eval = run_llamaindex_eval()
sql_eval.to_csv(f"./results/txt2sql/llamaindex.{lm.dict()['model']}.eval.tsv", sep="\t")


In [ ]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_db)
table_schema_objs = [(SQLTableSchema(table_name=t)) for t in tables] 

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_db, obj_index.as_retriever(similarity_top_k=3)
)

In [ ]:
sql_eval = run_llamaindex_eval()
sql_eval.to_csv(f"./results/txt2sql/llamaindex.{lm.dict()['model']}.eval.tsv", sep="\t")

In [ ]:
import ollama